In [1]:
import os
import pandas as pd
import numpy as np
import re
PUIdata = os.getenv('PUIDATA')

In [3]:
Complaints = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9')

In [4]:
#read columns to determine the data we need and what we can drop
Complaints.columns

Index([u'Unique Key', u'Created Date', u'Closed Date', u'Agency',
       u'Agency Name', u'Complaint Type', u'Descriptor', u'Location Type',
       u'Incident Zip', u'Incident Address', u'Street Name', u'Cross Street 1',
       u'Cross Street 2', u'Intersection Street 1', u'Intersection Street 2',
       u'Address Type', u'City', u'Landmark', u'Facility Type', u'Status',
       u'Due Date', u'Resolution Description',
       u'Resolution Action Updated Date', u'Community Board', u'Borough',
       u'X Coordinate (State Plane)', u'Y Coordinate (State Plane)',
       u'Park Facility Name', u'Park Borough', u'School Name',
       u'School Number', u'School Region', u'School Code',
       u'School Phone Number', u'School Address', u'School City',
       u'School State', u'School Zip', u'School Not Found',
       u'School or Citywide Complaint', u'Vehicle Type',
       u'Taxi Company Borough', u'Taxi Pick Up Location',
       u'Bridge Highway Name', u'Bridge Highway Direction', u'Road Ramp',

In [5]:
#drop columns we do not need
Complaints.drop([u'Created Date', u'Closed Date', u'Agency',
       u'Agency Name', u'Location Type', u'Incident Address', u'Street Name', u'Cross Street 1',
       u'Cross Street 2', u'Intersection Street 1', u'Intersection Street 2',
       u'Address Type', u'City', u'Landmark', u'Facility Type', u'Status',
       u'Due Date', u'Resolution Description',
       u'Resolution Action Updated Date',u'X Coordinate (State Plane)', u'Y Coordinate (State Plane)',
       u'Park Facility Name', u'Park Borough', u'School Name',
       u'School Number', u'School Region', u'School Code',
       u'School Phone Number', u'School Address', u'School City',
       u'School State', u'School Zip', u'School Not Found',
       u'School or Citywide Complaint', u'Vehicle Type',
       u'Taxi Company Borough', u'Taxi Pick Up Location',
       u'Bridge Highway Name', u'Bridge Highway Direction', u'Road Ramp',
       u'Bridge Highway Segment', u'Garage Lot Name', u'Ferry Direction',
       u'Ferry Terminal Name', u'Latitude', u'Longitude', u'Location'], axis=1, inplace=True)

In [6]:
Complaints.head()

,Unique Key,Complaint Type,Descriptor,Incident Zip,Community Board,Borough
0,31911011,Street Condition,Pothole,11224,13 BROOKLYN,BROOKLYN
1,31908754,Opinion for the Mayor,HOUSING,NaN,0 Unspecified,Unspecified
2,31910423,Root/Sewer/Sidewalk Condition,Trees and Sidewalks Program,11234,18 BROOKLYN,BROOKLYN
3,31909924,Illegal Parking,Blocked Hydrant,11218,12 BROOKLYN,BROOKLYN
4,31913310,Benefit Card Replacement,Medicaid,NaN,0 Unspecified,Unspecified


In [7]:
#we have 77 unique values, should be only 59 to represent all community boards 
len(Complaints['Community Board'].unique())

77

In [8]:
#there are 'unspecified' entries as well as entires for community boards that do not exist - 
#due possibly to a data entry error
Complaints['Community Board'].unique()

array(['13 BROOKLYN', '0 Unspecified', '18 BROOKLYN', '12 BROOKLYN',
       '02 STATEN ISLAND', '08 MANHATTAN', '06 BROOKLYN', '10 BRONX',
       '06 MANHATTAN', '04 BROOKLYN', '04 QUEENS', '09 MANHATTAN',
       '14 BROOKLYN', '05 MANHATTAN', '02 MANHATTAN', '10 QUEENS',
       '07 QUEENS', '02 QUEENS', '11 BROOKLYN', '05 QUEENS', '05 BROOKLYN',
       '09 QUEENS', '12 MANHATTAN', '01 BROOKLYN', '11 MANHATTAN',
       '03 BRONX', '03 STATEN ISLAND', '08 BROOKLYN', '03 BROOKLYN',
       '07 MANHATTAN', '10 MANHATTAN', '04 BRONX', '01 MANHATTAN',
       '07 BRONX', '09 BRONX', '03 MANHATTAN', '13 QUEENS', '02 BRONX',
       '12 QUEENS', '16 BROOKLYN', '08 QUEENS', '04 MANHATTAN',
       '14 QUEENS', '10 BROOKLYN', '11 QUEENS', '06 BRONX', '08 BRONX',
       '05 BRONX', '15 BROOKLYN', '02 BROOKLYN', '12 BRONX', '03 QUEENS',
       '06 QUEENS', '01 STATEN ISLAND', '01 BRONX', '01 QUEENS',
       '17 BROOKLYN', '11 BRONX', 'Unspecified QUEENS', '09 BROOKLYN',
       'Unspecified MANHATTAN'

In [32]:
#there is also one unspecified value within the borough field
Complaints['Borough'].unique()

array(['BROOKLYN', 'Unspecified', 'STATEN ISLAND', 'MANHATTAN', 'BRONX',
       'QUEENS'], dtype=object)

In [12]:
#drop duplicate values
print (len(Complaints))
dropuniquedupes = Complaints.drop_duplicates(['Unique Key'])

10187766


In [11]:
len(dropuniquedupes)

10187744

##### Removed duplicate entries for the 'Unique Key' which represented a number assigned to each call complaint. This removed only 22 entries that could have been data entry error in adding the same complaint information more than once or assigning a previously used unique key identifier for a new call. Removing duplicates from other columns would have certainly removed important data

In [13]:
#look for and sum NaN values
pd.isnull(dropuniquedupes).sum()

Unique Key              0
Complaint Type          0
Descriptor          31057
Incident Zip       794159
Community Board         0
Borough                 0
dtype: int64

##### The missing values that appear are for the descriptor and incident zip fields. This could be a result of not obtaining all of the information from a 311 caller or perhaps calls being ended before the representative can collect all of the information

##### I would not recommend using the complaint type field to fill in missing values for the descriptor field because there could be many different types of even/call descriptions under each complaint type. This would skew the results to indicate more types of calls or call descriptions than we can confidently say

In [36]:
#community districts do not exceed the number 18, those with a higher number, along with those marked as unspecified
#would be unplausible. Removing the unspecified values that appeared within the community board and borough fields

redact_311 = dropuniquedupes[dropuniquedupes['Community Board'].str.contains('Unspecfied') == False]
redact_311 = redact_311[redact_311['Borough'].str.contains('Unspecified') == False]



In [18]:
#open demographics .csv file as a variable called finaldemos
finaldemos = pd.read_csv(PUIdata + "/Final_Demographics.csv")

In [20]:
#determine column names, cd_id would stand for community district 
finaldemos.columns

Index([u'FIPS', u'cd_id', u'Total Population',
       u'Population Density (per sq. mile)', u'% Total Population: Male',
       u'% Total Population: 18 to 24 Years',
       u'% Total Population: 25 to 34 Years',
       u'% Total Population: 35 to 44 Years',
       u'% Population 5 Years And Over: Speak Only English',
       u'% Population 5 Years And Over: Spanish or Spanish Creole',
       ...
       u'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Production occupations',
       u'Employed Civilian Population 16 Years And Over: Production, transportation, and material moving  occupations: Transportation and material moving occupations',
       u'% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations',
       u'% Employed Civilian Population 16 Years And Over: Service occupations',
       u'% Employed Civilian Population 16 Years And Over: Sales and office occupations',
       

In [23]:
#determine the community districts that are listed in this column, there are 59
print(finaldemos['cd_id'].unique())
print (len(finaldemos['cd_id'].unique()))

['BX08' 'BX12' 'BX10' 'BX11' 'BX03' 'BX06' 'BX07' 'BX05' 'BX04' 'BX09'
 'BX01' 'BX02' 'MN12' 'MN09' 'MN10' 'MN11111' 'MN08' 'MN07' 'MN05' 'MN04'
 'MN06' 'MN03' 'MN01' 'MN02' 'SI03' 'SI02' 'SI01' 'BK01' 'BK04' 'BK03'
 'BK02' 'BK06' 'BK08' 'BK16' 'BK05' 'BK18' 'BK17' 'BK09' 'BK07' 'BK10'
 'BK12' 'BK14' 'BK15' 'BK11' 'BK13' 'QN01' 'QN03' 'QN07' 'QN11' 'QN13'
 'QN08' 'QN04' 'QN06' 'QN02' 'QN05' 'QN09' 'QN12' 'QN10' 'QN14']
59


In [28]:
#looking at variables to help determine who might be more likely to complain
demographcallers = finaldemos[['cd_id', "% Population 5 Years And Over: Speak Only English",
                               "% Population 5 Years And Over: Spanish or Spanish Creole" ]]
demographcallers.head()

,cd_id,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole
0,BX08,46.80,39.24
1,BX12,73.09,18.19
2,BX10,61.79,26.43
3,BX11,43.22,36.45
4,BX03,36.82,54.24


##### Community Districts where a higher percentage of people speak English may be more likely to have people call 311

In [37]:
#open infrastructure by community board data as a variable called Infrastruct
Infrastruct = pd.read_csv(PUIdata + "/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [38]:
#determining column names, not sure if it is geographic identifier or qualifying name, will inspect by .head function
Infrastruct.columns

Index([u'FIPS', u'Geographic Identifier', u'Qualifying Name', u'Households',
       u'Households: With An Internet Subscription',
       u'Households: Dial-Up Alone', u'Households: Dsl',
       u'Households: With Mobile Broadband',
       u'Households: Without Mobile Broadband', u'Households: Cable Modem',
       u'Households: With Mobile Broadband.1',
       u'Households: Without Mobile Broadband.1', u'Households: Fiber-Optic',
       u'Households: With Mobile Broadband.2',
       u'Households: Without Mobile Broadband.2',
       u'Households: Satellite Internet Service',
       u'Households: With Mobile Broadband.3',
       u'Households: Without Mobile Broadband.3',
       u'Households: Two or More Fixed Broadband Types, or Other',
       u'Households: With Mobile Broadband.4',
       u'Households: Without Mobile Broadband.4',
       u'Households: Mobile Broadband Alone or With Dialup',
       u'Households: Internet Access Without A Subscription',
       u'Households: No Internet Acc

In [39]:
#quailfying name represents community district
Infrastruct.head()

,FIPS,Geographic Identifier,Qualifying Name,Households,Households: With An Internet Subscription,Households: Dial-Up Alone,Households: Dsl,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: Cable Modem,...,Households: Mobile Broadband Alone or With Dialup,Households: Internet Access Without A Subscription,Households: No Internet Access,% Households: With An Internet Subscription,Households.1,Households: Has A Computer,Households: With Dial-Up Internet Subscription Alone,Households: With A Broadband Internet Subscription,Households: Without An Internet Subscription,Households: No Computer
0,3603701,79500US3603701,"NYC-Bronx Community District 8--Riverdale, New...",42035,31795,444,2813,946,1867,19178,...,2168,2119,8121,75.64,42035,35048,404,30943,3701,6987
1,3603702,79500US3603702,"NYC-Bronx Community District 12--Wakefield, Ne...",44830,32243,178,849,405,444,18653,...,928,1891,10696,71.92,44830,36700,178,31435,5087,8130
2,3603703,79500US3603703,"NYC-Bronx Community District 10--Co-op City, N...",47050,32729,158,1863,398,1465,20044,...,639,2882,11439,69.56,47050,38700,158,32333,6209,8350
3,3603704,79500US3603704,NYC-Bronx Community District 11--Pelham Parkwa...,44922,32003,141,1478,474,1004,17917,...,1001,2722,10197,71.24,44922,37237,122,31278,5837,7685
4,3603705,79500US3603705,"NYC-Bronx Community District 3 & 6--Belmont, N...",57556,35503,88,1297,651,646,15847,...,1385,3312,18741,61.68,57556,42576,88,33408,9080,14980


In [42]:
#determine the community districts that are listed in this column, there are 55
print(Infrastruct['Qualifying Name'].unique())
print (len(Infrastruct['Qualifying Name'].unique()))

['NYC-Bronx Community District 8--Riverdale, New York'
 'NYC-Bronx Community District 12--Wakefield, New York'
 'NYC-Bronx Community District 10--Co-op City, New York'
 'NYC-Bronx Community District 11--Pelham Parkway, New York'
 'NYC-Bronx Community District 3 & 6--Belmont, New York'
 'NYC-Bronx Community District 7--Bedford Park, New York'
 'NYC-Bronx Community District 5--Morris Heights, New York'
 'NYC-Bronx Community District 4--Concourse, New York'
 'NYC-Bronx Community District 9--Castle Hill, New York'
 'NYC-Bronx Community District 1 & 2--Hunts Point, New York'
 'NYC-Manhattan Community District 12--Washington Heights, New York'
 'NYC-Manhattan Community District 9--Hamilton Heights, New York'
 'NYC-Manhattan Community District 10--Central Harlem PUMA, New York'
 'NYC-Manhattan Community District 11--East Harlem PUMA, New York'
 'NYC-Manhattan Community District 8--Upper East Side PUMA, New York'
 'NYC-Manhattan Community District 7--Upper West Side & West Side PUMA, New York'

In [45]:
#looking at variables to help determine information on internet/broadband useage by Community District
Infrastructinfo = Infrastruct[['Qualifying Name', 'Households: With Mobile Broadband',
                               'Households: Without Mobile Broadband', 'Households: With Mobile Broadband.1',
                               'Households: With Mobile Broadband.2', 'Households: With Mobile Broadband.3', 
                              'Households: With Mobile Broadband.4']]
Infrastructinfo.head()

,Qualifying Name,Households: With Mobile Broadband,Households: Without Mobile Broadband,Households: With Mobile Broadband.1,Households: With Mobile Broadband.2,Households: With Mobile Broadband.3,Households: With Mobile Broadband.4
0,"NYC-Bronx Community District 8--Riverdale, New...",946,1867,10433,433,37,3510
1,"NYC-Bronx Community District 12--Wakefield, Ne...",405,444,5577,2358,0,2146
2,"NYC-Bronx Community District 10--Co-op City, N...",398,1465,6377,1200,0,3450
3,NYC-Bronx Community District 11--Pelham Parkwa...,474,1004,5624,2272,241,2137
4,"NYC-Bronx Community District 3 & 6--Belmont, N...",651,646,6690,695,111,6760


In [ ]:
# what is low and high internet connections